# Outlook

This notebook is designed to understand how to use a gymnasium environment as a BBRL agent in practice, using autoreset=False.
It is part of the [BBRL documentation](https://github.com/osigaud/bbrl/docs/index.html).

If this is your first contact with BBRL, you may start be having a look at [this more basic notebook](01-basic_concepts.student.ipynb).

## Installation and Imports

The BBRL library is [here](https://github.com/osigaud/bbrl).

Below, we import standard python packages, pytorch packages and gymnasium environments.

In [1]:
# Installs the necessary Python and system libraries
try:
    from easypip import easyimport, easyinstall, is_notebook
except ModuleNotFoundError as e:
    get_ipython().run_line_magic("pip", "install easypip")
    from easypip import easyimport, easyinstall, is_notebook

easyinstall("bbrl>=0.2.2")
easyinstall("swig")
easyinstall("bbrl_gymnasium>=0.2.0")
easyinstall("bbrl_gymnasium[classic_control]")

In [2]:
import os
import sys
from pathlib import Path
import math

from moviepy.editor import ipython_display as video_display
import time
from tqdm.auto import tqdm
from typing import Tuple, Optional
from functools import partial

from omegaconf import OmegaConf
import torch
import bbrl_gymnasium

import copy
from abc import abstractmethod, ABC
import torch.nn as nn
import torch.nn.functional as F
from time import strftime
OmegaConf.register_new_resolver(
    "current_time", lambda: strftime("%Y%m%d-%H%M%S"), replace=True
)

In [3]:
# Imports all the necessary classes and functions from BBRL
from bbrl.agents.agent import Agent
from bbrl import get_arguments, get_class, instantiate_class
# The workspace is the main class in BBRL, this is where all data is collected and stored
from bbrl.workspace import Workspace

# Agents(agent1, agent2, agent3, ...) executes the different agents the one after the other
# TemporalAgent(agent) executes an agent over multiple timesteps in the workspace,
# or until a given condition is reached

from bbrl.agents import Agents, TemporalAgent
from bbrl.agents.gymnasium import ParallelGymAgent, make_env

In [4]:
from gymnasium.wrappers.time_limit import TimeLimit

## Definition of agents

We first create an Agent representing [the CartPole-v1 gym environment](https://gymnasium.farama.org/environments/classic_control/cart_pole/).
This is done using the [ParallelGymAgent](https://github.com/osigaud/bbrl/blob/40fe0468feb8998e62c3cd6bb3a575fef88e256f/src/bbrl/agents/gymnasium.py#L261) class.

The ParallelGymAgent is an agent able to execute a batch of gymnasium environments
with or without auto-resetting. These agents produce multiple variables in the workspace:
’env/env_obs’, ’env/reward’, ’env/timestep’, ’env/terminated’,
'env/truncated', 'env/done', ’env/cumulated_reward’.

When called at timestep t=0, the environments are automatically reset. At
timestep t>0, these agents will read the ’action’ variable in the workspace at
time t − 1 to generate the next state, by calling the step(action) of the contained gymnasium environment.

In the example below, we are working with batches (i.e. several episodes at the same time),
so here our agent uses `n_envs = 3` environments.

In [9]:
# We run episodes over 3 environments at a time
n_envs = 3
env_agent = ParallelGymAgent(partial(make_env, 'CartPole-v1', autoreset=False, wrappers=[lambda x: TimeLimit(x,5)]), n_envs, reward_at_t=False)
# The random seed is set to 2139
env_agent.seed(2139)

obs_size, action_dim = env_agent.get_obs_and_actions_sizes()
print(f"Environment: observation space in R^{obs_size} and action space {{1, ..., {action_dim}}}")

Environment: observation space in R^4 and action space {1, ..., 2}


In [11]:
# Creates a new workspace
workspace = Workspace()

# Execute the first step
env_agent(workspace, t=0)

# Our first set of observations. The size of the observation space is 4, and we have 3 environments.
obs = workspace.get("env/env_obs", 0)
print("Observation", obs)



Observation tensor([[ 0.0124, -0.0116,  0.0219,  0.0477],
        [ 0.0100, -0.0009, -0.0109, -0.0069],
        [ 0.0457,  0.0205,  0.0487,  0.0031]])


To generate more steps into the workspace, we need to send actions to the environment.

### Random action without agent

We first set an action directly without using an agent

In [12]:
# Sets the next action
action = torch.randint(0, action_dim, (n_envs, ))
workspace.set("action", 0, action)
print(action)
env_agent(workspace, t=1)

# And perform one step
workspace.get("env/env_obs", 1)

tensor([1, 1, 1])


tensor([[ 0.0122,  0.1832,  0.0228, -0.2380],
        [ 0.0100,  0.1943, -0.0111, -0.3030],
        [ 0.0461,  0.2149,  0.0488, -0.2739]])

Let us now look at what's in the workspace. You can see below all the variables it generates.

In [13]:
for key in workspace.variables.keys():
    print(key, workspace[key])

env/env_obs tensor([[[ 0.0124, -0.0116,  0.0219,  0.0477],
         [ 0.0100, -0.0009, -0.0109, -0.0069],
         [ 0.0457,  0.0205,  0.0487,  0.0031]],

        [[ 0.0122,  0.1832,  0.0228, -0.2380],
         [ 0.0100,  0.1943, -0.0111, -0.3030],
         [ 0.0461,  0.2149,  0.0488, -0.2739]]])
env/terminated tensor([[False, False, False],
        [False, False, False]])
env/truncated tensor([[False, False, False],
        [False, False, False]])
env/done tensor([[False, False, False],
        [False, False, False]])
env/reward tensor([[0., 0., 0.],
        [1., 1., 1.]])
env/cumulated_reward tensor([[0., 0., 0.],
        [1., 1., 1.]])
env/timestep tensor([[0, 0, 0],
        [1, 1, 1]])
action tensor([[1, 1, 1]])


You can observe that we have two time steps for each variable that are stored
within tensors where the first dimension is time.

You can also see that by convention, all variables written by the environment start with "env/".

### Random agent

The process above can be
automatized with `Agents` and `TemporalAgent` as shown below - but first we have
to create an agent that selects the actions (here, randomly).

In [14]:
class RandomAgent(Agent):
    def __init__(self, action_dim):
        super().__init__()
        self.action_dim = action_dim

    def forward(self, t: int, choose_action=True, **kwargs):
        """An Agent can use self.workspace"""
        obs = self.get(("env/env_obs", t))
        action = torch.randint(0, self.action_dim, (len(obs), ))
        self.set(("action", t), action)

# Each agent is run in the order given when constructing Agents
agents = Agents(env_agent, RandomAgent(action_dim))

# And the TemporalAgent allows to run through time
t_agents = TemporalAgent(agents)

In [15]:
# We can now run the agents throught time with a simple call...

workspace = Workspace()
t_agents(workspace, t=0, stop_variable="env/done", stochastic=True)

In [16]:
for key in workspace.variables.keys():
    print(key, workspace[key])

env/env_obs tensor([[[ 0.0351, -0.0244,  0.0483,  0.0200],
         [-0.0070,  0.0379, -0.0248,  0.0201],
         [-0.0378,  0.0325,  0.0493, -0.0067]],

        [[ 0.0346,  0.1700,  0.0487, -0.2571],
         [-0.0062, -0.1568, -0.0244,  0.3049],
         [-0.0372, -0.1633,  0.0492,  0.3012]],

        [[ 0.0380, -0.0258,  0.0435,  0.0505],
         [-0.0093, -0.3516, -0.0183,  0.5898],
         [-0.0404, -0.3591,  0.0552,  0.6089]],

        [[ 0.0375,  0.1687,  0.0445, -0.2281],
         [-0.0164, -0.5464, -0.0065,  0.8766],
         [-0.0476, -0.1648,  0.0674,  0.3341]],

        [[ 0.0409,  0.3632,  0.0400, -0.5064],
         [-0.0273, -0.7415,  0.0110,  1.1672],
         [-0.0509,  0.0293,  0.0741,  0.0635]],

        [[ 0.0481,  0.1675,  0.0298, -0.2014],
         [-0.0421, -0.9367,  0.0343,  1.4633],
         [-0.0503,  0.2233,  0.0754, -0.2050]]])
env/terminated tensor([[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False,

### Termination

`env/done` tells us whether the episode was finished or not (it is either terminated or truncated)
here, with NoAutoReset, we wait that all episodes are "done"
and when the episode is finished, the variables are copied for that environment until all episodes are done.
So, when an environment is done before the others, its content is copied until the termination of all environments.
This is convenient for collecting the final reward.

In [18]:
workspace["env/done"].shape, workspace["env/done"][-10:]

(torch.Size([6, 3]),
 tensor([[False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [ True,  True,  True]]))

You can see that the variable is copied until all episodes are done.

### Observations

The resulting tensor of observations, with the last two observations:

In [19]:
workspace["env/env_obs"].shape, workspace["env/env_obs"][-2:]

(torch.Size([6, 3, 4]),
 tensor([[[ 0.0409,  0.3632,  0.0400, -0.5064],
          [-0.0273, -0.7415,  0.0110,  1.1672],
          [-0.0509,  0.0293,  0.0741,  0.0635]],
 
         [[ 0.0481,  0.1675,  0.0298, -0.2014],
          [-0.0421, -0.9367,  0.0343,  1.4633],
          [-0.0503,  0.2233,  0.0754, -0.2050]]]))

### Rewards

The resulting tensor of rewards, with the last 8 rewards:

In [20]:
workspace["env/reward"].shape, workspace["env/reward"][-8:]

(torch.Size([6, 3]),
 tensor([[0., 0., 0.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]))

and the cumulated rewards:

In [21]:
workspace["env/cumulated_reward"].shape, workspace["env/cumulated_reward"][-8:]

(torch.Size([6, 3]),
 tensor([[0., 0., 0.],
         [1., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.],
         [4., 4., 4.],
         [5., 5., 5.]]))

### Actions

The resulting tensor of actions, with the last two actions:

In [22]:
workspace["action"].shape, workspace["action"][-2:]

(torch.Size([6, 3]),
 tensor([[0, 0, 1],
         [1, 1, 0]]))

## Exercise

Create a stupid agent that always outputs action 1, until the episode stops.
Watch the content of the resulting workspace.

In [23]:
class DummyAgent(Agent):
    def __init__(self, action_dim):
        super().__init__()
        self.action_dim = action_dim
        
    def forward(self, t: int, choose_action=True, **kwargs):
        obs = self.get(("env/env_obs", t))
        action = torch.ones((len(obs), ), dtype=torch.int)
        self.set(("action", t), action)

agents = Agents(env_agent, DummyAgent(action_dim))

t_agents = TemporalAgent(agents)

workspace = Workspace()
t_agents(workspace, t=0, stop_variable="env/done", stochastic=True)

for key in workspace.variables.keys():
    print(key, workspace[key])

env/env_obs tensor([[[ 1.5349e-02, -1.6139e-02, -1.4235e-02,  2.6596e-02],
         [-2.9842e-02,  4.2431e-02,  7.2186e-04,  8.1762e-03],
         [-8.3105e-03,  6.8620e-03,  1.7204e-02, -1.3014e-02]],

        [[ 1.5026e-02,  1.7918e-01, -1.3703e-02, -2.7054e-01],
         [-2.8993e-02,  2.3754e-01,  8.8538e-04, -2.8428e-01],
         [-8.1733e-03,  2.0173e-01,  1.6943e-02, -3.0022e-01]],

        [[ 1.8610e-02,  3.7450e-01, -1.9114e-02, -5.6752e-01],
         [-2.4242e-02,  4.3265e-01, -4.8002e-03, -5.7668e-01],
         [-4.1386e-03,  3.9661e-01,  1.0939e-02, -5.8751e-01]],

        [[ 2.6100e-02,  5.6988e-01, -3.0465e-02, -8.6616e-01],
         [-1.5589e-02,  6.2784e-01, -1.6334e-02, -8.7087e-01],
         [ 3.7936e-03,  5.9158e-01, -8.1133e-04, -8.7673e-01]],

        [[ 3.7497e-02,  7.6541e-01, -4.7788e-02, -1.1683e+00],
         [-3.0326e-03,  8.2318e-01, -3.3751e-02, -1.1686e+00],
         [ 1.5625e-02,  7.8671e-01, -1.8346e-02, -1.1697e+00]],

        [[ 5.2806e-02,  9.6112e-0